In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
dat = pd.read_csv('/content/drive/MyDrive/Deep Learning/features_30_sec.csv')
dat


In [ ]:
import os
import librosa
import IPython.display as ipd

dir = '/content/drive/MyDrive/Deep Learning/genres_original'
for i in os.listdir(dir):
  aud,sample_rate = librosa.load(dir + '/' + i)
  librosa.display.waveplot(aud,sr=sample_rate)
  wn = ipd.Audio(dir + '/' + i)
  ipd.display(wn)
  break


In [ ]:
print(aud)
print(np.shape(aud))
print(sample_rate)

In [ ]:
from librosa.feature import mfcc
op = mfcc(y=aud,sr=sample_rate,n_mfcc=40)
print(np.shape(op))
op

In [ ]:
feature = np.mean(np.transpose(op),axis=0)
feature

In [ ]:
X = []
c=0
for i in os.listdir(dir):
  aud,sample_rate = librosa.load(dir + '/' + i)
  op = mfcc(y=aud,sr=sample_rate,n_mfcc=40)
  feature = np.mean(np.transpose(op),axis=0)
  X.append(feature)
  if(c==499):
    break
  c=c+1


In [ ]:
X = np.array(X)
print(np.shape(X))
X

In [ ]:
y = dat['label'][0:500]
y

In [ ]:
y = pd.get_dummies(y)
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
print(np.shape(X_train))
print(np.shape(y_train))
print(np.shape(X_test))
print(np.shape(y_test))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten

model = Sequential()
model.add(Flatten())
model.add(Dense(units=100, input_dim=40, activation='relu'))
model.add(Dense(units=500, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test))

In [ ]:
y_pred = model.predict(X_test)
print(y_pred)
for i in range(0,np.shape(y_pred)[0]):
  for j in range(0,np.shape(y_pred)[1]):
    if(y_pred[i][j]>0.5):
      y_pred[i][j]=1
    else:
      y_pred[i][j]=0

y_pred

In [ ]:
y_test

In [ ]:
c=0
y_test=np.array(y_test)
for i in range(0,np.shape(y_pred)[0]):
  for j in range(0,np.shape(y_pred)[1]):
    if(y_test[i][j]==1 and y_pred[i][j]==1):
      c=c+1
print(c/100)


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)